In [2]:
import json
import tqdm
import random
import evaluate
import pandas as pd
from datasets import load_dataset

/home/gcilleru/ProphetNet/GENIE/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")


In [26]:
gen_epoch = 1
df = []
for model_mask, num_ckpt in zip(
    [
        "human", "human", "human", "30", "30", "30"
    ], 
    [
        "20000","80000", "120000","20000","80000", "120000",
    ]):

        # test tgt
        # tgt_file = f"recipes/mask_{model_mask}/org_data/test.tgt"
        tgt_file = f"data/eval/org_data/test.tgt"
        gen_file = f"data/mask_{model_mask}/gen_{num_ckpt}/rank0_gen_seed_101_num1_epoch{gen_epoch}.txt"
        # load data into 2 lists and assert the have the same amount of lines
        with open(tgt_file, "r") as f:
            tgt_lines = f.readlines()
        with open(gen_file, "r") as f:
            gen_lines = f.readlines()
        assert len(tgt_lines) == len(gen_lines)
        rouge_metric = rouge.compute(predictions = tgt_lines, references=gen_lines)
        df.append({
             "mask": model_mask,
                "num_steps": num_ckpt,
                "bleu":bleu.compute(predictions = tgt_lines, references=gen_lines)["bleu"],
                "rouge1": rouge_metric["rouge1"],
                "rouge2": rouge_metric["rouge2"],
                "rougeL": rouge_metric["rougeL"],
        })

for steps in [ "80000", "120000", "180000"]:
    with open(f"data/eval/gpt2/test-{steps}.gen", "r") as f:
        preds = f.readlines()
    preds = [p.strip() for p in preds if p.strip() != ""]
    tgt_file = f"data/eval/org_data/test.tgt"
    with open(tgt_file, "r") as f:
        tgt_lines = f.readlines()
    assert len(tgt_lines) == len(preds)
    rouge_metric = rouge.compute(predictions = preds, references=tgt_lines)
    df.append({
         "mask": "gpt2",
            "num_steps": steps,
            "bleu":bleu.compute(predictions = preds, references=tgt_lines)["bleu"],
            "rouge1": rouge_metric["rouge1"],
            "rouge2": rouge_metric["rouge2"],
            "rougeL": rouge_metric["rougeL"],
    })
df = pd.DataFrame(df)
df


INFO:absl:Using default tokenizer.


INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.


,mask,num_steps,bleu,rouge1,rouge2,rougeL
0,human,20000,0.046564,0.259223,0.078110,0.207200
1,human,80000,0.059962,0.311585,0.104963,0.245962
2,human,120000,0.058203,0.302225,0.087300,0.244912
3,30,20000,0.034335,0.281050,0.067159,0.215390
4,30,80000,0.048419,0.292954,0.079378,0.227571
5,30,120000,0.051525,0.297781,0.089028,0.236642
6,gpt2,80000,0.025535,0.218769,0.059123,0.155956
7,gpt2,120000,0.031360,0.231239,0.068221,0.169918
8,gpt2,180000,0.029417,0.225583,0.070260,0.170000


In [ ]:
# use evaluate to get bleu and rouge scores


In [ ]:
# load masked_recipes from json
with open("recipes/raw/masked_recipes.json", "r") as f:
    masked_recipes = json.load(f)
random.seed(101)
# split dataset into train, valid, test
random.shuffle(masked_recipes)
test = masked_recipes[int(len(masked_recipes)*0.95):]

In [ ]:
dataset = load_dataset("recipe_nlg", data_dir="recipes/raw/dataset")
test_ids = [recipe["id"] for recipe in test[300:400]]

In [ ]:
len(test_ids)

In [ ]:
test_data = []
for i in test_ids:
    test_data.append(dataset["train"][i])

In [ ]:
test_genie = []
test_gpt2 = []
targets = []
for sample in test_data:
    recipe = sample["title"] + " Ingredients: " + " ".join(sample["ingredients"]) + " Directions: -" + " -".join(sample["directions"][:-3])
    tgt = "-"+" -".join(sample["directions"][-3:])
    test_genie.append(recipe+ " [MASK]")
    test_gpt2.append(recipe)
    targets.append(tgt)
    

In [ ]:
# save to folder data/eval separated by \n
with open("data/eval/test.src", "w") as f:
    f.write("\n".join(test_genie))
with open("data/eval/test.tgt", "w") as f:
    f.write("\n".join(targets))
with open("data/eval/test.gpt2", "w") as f:
    f.write("\n".join(test_gpt2))

In [24]:
# load got2 checkpoints from recipes/gpt2/outputs/checkpoint-80000
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
# load test src from data/eval/test.gpt2 and test tgt from data/eval/test.tgt
with open("data/eval/org_data/test.gpt2", "r") as f:
    test_src = f.readlines()
with open("data/eval/org_data/test.tgt", "r") as f:
    test_tgt = f.readlines()

steps = "80000"
for steps in [ "80000", "120000", "180000"]:
    model = GPT2LMHeadModel.from_pretrained(f"recipes/gpt2/outputs/checkpoint-{steps}")
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    model.cuda(3)
    preds = []
    for src in tqdm.tqdm(test_src, total=len(test_src), desc=f"steps: {steps}"):
        src = src + " -"
        input_ids = tokenizer.encode(src, return_tensors="pt").to(3)
        # stop otken id is 198
        outputs = model.generate(input_ids, max_length = 512, num_beams=5, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True, eos_token_id=198, pad_token_id=tokenizer.eos_token_id)
        pred = tokenizer.decode(outputs[0][len(tokenizer.encode(src)):])
        preds.append(pred)
    # save to data/eval/gpt2/test.gen
    with open(f"data/eval/gpt2/test-{steps}.gen", "w") as f:
        f.write("\n".join(preds))


steps: 180000: 100%|██████████| 100/100 [02:36<00:00,  1.57s/it]


In [1]:
for steps in [ "80000", "120000", "180000"]:
    # load the gpt2 gen file, delete all empty lines and rewrite it
    with open(f"data/eval/gpt2/test-{steps}.gen", "r") as f:
        preds = f.readlines()
    preds = [p.strip() for p in preds if p.strip() != ""]
    with open(f"data/eval/gpt2/test-{steps}.gen", "w") as f:
        f.write("\n".join(preds))

In [7]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
steps = "120000"
model = GPT2LMHeadModel.from_pretrained(f"recipes/gpt2/outputs/checkpoint-{steps}")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model.cuda(3)
src = "Mandarin Spinach Salad Directions: -Mix 2 tablespoons sugar with small amount of water over low heat until sugar has melted. -Add almonds and stir until well coated. -Cook, then break apart. -Store at room temperature. -In large bowl, place lettuce, spinach, celery and onions. Immediately before serving, toss with Sweet-Sour Dressing. -Then add mandarin oranges and almonds. -Don't add Sweet-Sour Dressing until ready to serve; it will make the salad soggy. Ingredients:"
input_ids = tokenizer.encode(src, return_tensors="pt").to(3)
# stop otken id is 198
outputs = model.generate(input_ids, max_length = 512, num_beams=5, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True, eos_token_id=198, pad_token_id=tokenizer.eos_token_id)
pred = tokenizer.decode(outputs[0][len(tokenizer.encode(src)):])
pred


' 1 (10 oz.) pkg. frozen chopped spinach 1/2 c. mayonnaise 1 Tbsp. lemon juice 1 tsp. grated onion 2 hard-boiled eggs, chopped 1 small can sliced water chestnuts, drained and sliced 1 can cream of chicken soup, undiluted salt and pepper to taste paprika to sprinkle on top (if desired) 1 to 2 slices American cheese slices, cut in strips (for garnish) (optional) 2 toasted English muffins, halved (toasted, if you like) or bread croutons or crackers, crumbled (garnish with butter or margarine or butter (as a topping) for topping (can be used as a salad dressing for the top of salad, optional) mayo for dressing, as topping, for salad or salad (mayo or dressing or topping for sandwiches, etc.\n'

In [ ]:
# generate with max-length 64 and beam search 5 all the predictions from test_src
test_gen = []
for src in tqdm.tqdm(test_src):
    input_ids = tokenizer.encode(src, return_tensors="pt")
    outputs = model.generate(input_ids, num_beams=5, num_return_sequences=5, no_repeat_ngram_size=2, early_stopping=True)
    for output in outputs:
        test_gen.append(tokenizer.decode(output, skip_special_tokens=True))

In [ ]:
# save test_gen to data/eval/gpt2/gen_80000.txt
with open("data/eval/gpt2/gen_80000.txt", "w") as f:
    f.write("\n".join(test_gen))

In [ ]:
test_gpt2[0]